## Image Feature Extraction

This notebook reads a set of images stored in a directory and extracts image features from a pre-trained network with FastAI. Required fastai==2.5.2 with CUDA enabled.

In [2]:
# import libraries
import shutil, timm, time
from tqdm import tqdm
from pprint import pprint
from timm import create_model
from fastai.vision.all import *

## Functions

In [3]:
# create dataloader from dataframe
def create_dls(df, size=224, x_col="path", y_col="labels", col_splitter=True, bs=128):
  item_tfms = [Resize(size)]
  batch_tfms = []
  if col_splitter:
    splitter = ColSplitter("is_valid")
  else:
    splitter = RandomSplitter(valid_pct=0.0, seed=36)
  # Create datablock
  rocks = DataBlock(
              blocks = (ImageBlock, CategoryBlock),
              get_x = ColReader(x_col),
              splitter = splitter,
              get_y = ColReader(y_col),
              item_tfms = item_tfms,
              batch_tfms = batch_tfms
          )
  dls = rocks.dataloaders(df,bs=bs, shuffle=False, drop_last=False)
  return dls

# define input and output functions
def get_output(module, input_value, output):
  return output.flatten(1)
def get_input(module, input_value, output):
  return list(input_value)[0]

# get layer named: name
def get_named_module_from_model(model, name):
  for n, m in model.named_modules():
    if n == name:
      return m
  return None

# hook to last fully connect layer
class Hook():
  "Create a hook on `m` with `hook_func`."
  def __init__(self, m:nn.Module, hook_func, is_forward:bool=True, detach:bool=True):
    self.hook_func,self.detach,self.stored = hook_func,detach,None
    f = m.register_forward_hook if is_forward else m.register_backward_hook
    self.hook = f(self.hook_fn)
    self.removed = False

  def hook_fn(self, module:nn.Module, input, output):
    "Applies `hook_func` to `module`, `input`, `output`."
    if self.detach:
      input  = (o.detach() for o in input ) if is_listy(input ) else input.detach()
      output = (o.detach() for o in output) if is_listy(output) else output.detach()
    self.stored = self.hook_func(module, input, output)

  def remove(self):
    "Remove the hook from the model."
    if not self.removed:
      self.hook.remove()
      self.removed=True

  def __enter__(self, *args): return self
  def __exit__(self, *args): self.remove()

## Read Data & Set Output Path

In [18]:
# set output pickle and csv paths
csv_path = r'../data/processed2/feats_MSDP_features.csv'
pkl_path = r'../data/processed2/feats_MSDP_features.pkl'

# set paths to image tiles
img_path = r'../data/processed2/tiles'

# read images paths into data frame 
sublists= [[os.path.join(i[0], j) for j in i[2] if j.endswith('.jpg') ] for i in os.walk(img_path)]
filelist = np.array([item for sublist in sublists for item in sublist])
df = pd.DataFrame()
df['img'] = [x.split('/')[-1] for x in filelist]
df['path'] = [os.path.join(img_path, x) for x in filelist]
df['labels'] = [x.split('/')[8] for x in filelist]
df.head()

Define batch size and create a dataloader.

In [ ]:
# define batch size
bs = 4

# create dataloader for feature extraction
dls = create_dls(df,col_splitter=False, bs=bs)

## Define model

We use timm to get SoTA trained model
<https://github.com/rwightman/pytorch-image-models/>

In [ ]:
# list model names
model_names = timm.list_models('*efficientnetv2*',pretrained=True)
# pprint(model_names)

In [ ]:
# define body
base_model_name = 'efficientnetv2_rw_s'

# get the body
body = create_model(base_model_name, pretrained=True, in_chans=3, num_classes=0)
new_body = nn.Sequential(*list(body.children())[:-2])
nf = num_features_model(nn.Sequential(*body.children())) 

# define head and model
head = create_head(nf, dls.c)
model = nn.Sequential(new_body, head)

# initialise weight and create learner
apply_init(model[1], nn.init.kaiming_normal_)
learner = Learner(dls, model, loss_func=LabelSmoothingCrossEntropy(), metrics=accuracy)

# get model and copy it to GPU
model = learner.model
model.cuda()
linear_output_layer = get_named_module_from_model(model, "1.4")
print("Finished prepare model!")

Finished prepare model!


## Run Feature Extraction

In [ ]:
# run feature extraction procedure
img_repr_map = {}
start_time = time.time()
print("Generating feature map ...")
total = 0
with Hook(linear_output_layer, get_output, True, True) as hook:
    start = time.time()
    for i, (xb, yb) in enumerate(dls[0]):
        cur_bs = xb.shape[0]
        end = i*bs + cur_bs
        img_ids = dls[0].items[i*bs:end].path.values
        result = model.eval()(xb)
        img_reprs = hook.stored.cpu().numpy()
        img_reprs = img_reprs.reshape(cur_bs, -1)
        for j in range(cur_bs):
            img_repr_map[img_ids[j]] = img_reprs[j]
        if(len(img_repr_map) % 12800 == 0):
            end = time.time()
            print(f'{end-start} secs for 12800 images')
            start = end

# place features in a data frame
print("Total processing time: ", time.time()-start_time)
df_feature = pd.DataFrame(img_repr_map.items(), columns=['path', 'feature'])

Generating feature map ...
137.40247082710266 secs for 12800 images
135.29288482666016 secs for 12800 images
141.77030682563782 secs for 12800 images
157.8251769542694 secs for 12800 images
163.55647158622742 secs for 12800 images
146.89552187919617 secs for 12800 images
145.5284366607666 secs for 12800 images
144.8761203289032 secs for 12800 images
147.19988894462585 secs for 12800 images
145.5785939693451 secs for 12800 images
148.4882583618164 secs for 12800 images
146.3279824256897 secs for 12800 images
145.01856541633606 secs for 12800 images
145.18116807937622 secs for 12800 images
Total processing time:  2158.337628364563


Save features to pickle and csv.

In [ ]:
# save the feature data frame to pickle
df_feature.to_pickle(pkl_path, protocol=4)

# add prefix 'X' to features and save to csv
df_wide = pd.DataFrame(df_feature["feature"].to_list())
df_wide = df_wide.add_prefix('X')
df_wide.insert(0, "path", df_feature["path"])
df_wide.to_csv(csv_path)